In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from pandas import DataFrame, Series
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm


C:\Users\never\Anaconda3\envs\tensorflow35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
rawdata = pd.read_csv("블랑1664_75.csv",engine="python",names=["text"],encoding = "cp949",skiprows=1)

In [6]:
rawdata.head()

,text
0,혹시 1664블랑 맥주 먹어본사람있음? 먼맛이야
1,블랑1664 진짜존맛이야ㅜ맥주지린내가 안나
2,아ㅠ 기린 탈 것 얘기하시는 줄 알았자나요... 저는 블랑1664 조아해요. 아
3,갓 블랑 1664
4,블랑 1664 드세요 내 최애 맥주야


In [7]:
#원본 데이터셋의 리뷰 수

len(rawdata["text"])

373

In [8]:
rawdata["text"][rawdata["text"].duplicated()].head(20)

74    0
75    0
76    0
77    0
78    0
79    0
80    0
81    0
82    0
83    0
84    0
85    0
86    0
87    0
88    0
89    0
90    0
91    0
92    0
93    0
Name: text, dtype: object

In [9]:
#중복값 제거 후의 리뷰 수

len(pd.unique(rawdata["text"]))

74

In [15]:
#중복값 제거

text = rawdata["text"].unique()

In [13]:
#결측값 제거

text = rawdata["text"].dropna()

In [14]:
#twitter 형태소분석기를 함수로 정의

from konlpy.tag import Okt
twitter_tag = Okt()

def vect_tokenizer(text):
    return twitter_tag.morphs(text)

In [20]:
#twitter 형태소 분석기를 통해 각 문장을 형태소로 나눈다.

morph = list()

for i in text:
    raw = twitter_tag.morphs(i)
    raw = [raw[k] for k in np.where([len(j) != 1 for j in raw])[0]]
    morph.append(raw)

In [27]:
#문장 길이의 평균, 표준편차를 구한다.

a = 0
b = 0

for i in morph:
    a += len(i)

mu = a/len(morph)

for i in morph:
    b += (len(i) - mu)**2

std_dev = (b/len(morph))**0.5

print(mu,std_dev)

#평균에서 1 표준편차만큼을 더한 값을 문장 길이로 설정한다.

NameError: name 'vectorized' is not defined

In [273]:
#앞서 살펴본 문장길이를 maxlen에 넣어준다.

from keras.preprocessing import sequence

morph_seq = sequence.pad_sequences(morph,maxlen=50)

In [22]:
#Word2Vec 에 앞서 분석한 형태소를 집어넣고 돌린다.
## size = 투입되는 문장의 크기, window = 중심단어에서 앞 뒤로 확인할 단어의 갯수, worker = 컴퓨터 코어의 수, iter = 반복횟수

from gensim.models import Word2Vec
embedding_model2 = Word2Vec(morph_seq, size=50, window = 50, workers=4, iter=100, sg=1)

C:\Users\never\Anaconda3\envs\tensorflow35\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [ ]:
### 아래는 글자로 되어있는 문장을 숫자로 바꿔주는 벡터화 함수임

In [169]:
vect = CountVectorizer(tokenizer = vect_tokenizer)

vect.fit(text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<function vect_tokenizer at 0x000001FBB91C8268>,
        vocabulary=None)

In [219]:
vectorized = list()

for k in morph:
    raw = list()
    for i in k:
        try:
            raw.append(vect.vocabulary_[i])
        except:
            pass
    vectorized.append(raw)

why = DataFrame(vectorized)

why.to_csv("벡터화.csv")